In [1]:
from typing import Tuple
import re
import json
from collections import defaultdict, OrderedDict

In [2]:
import numpy as np
import pandas as pd

## Read namuwiki dump

In [3]:
with open('../data/namuwiki.json') as f:
    context_ = json.load(f)

In [4]:
context = defaultdict(dict)
for doc in context_:
    context[doc['title']][doc['namespace']] = doc

## Parser

In [6]:
regex_document = re.compile('\[\[(.[^:]+?)\]\]')
regex_table = re.compile('(?<=\|\|)(.*)(?=\|\|)')
regex_bracket = re.compile('\((.+?)\)')
regex_redirect = re.compile('#redirect (.+?)')

regex_tags = OrderedDict({
    'horizontal_line': ('', re.compile('\-{4,9}')),
    'comment': ('', re.compile('##\s?(.*)')),
    
    'header': (r'\1', re.compile('\={2,6}\#?(.+?)\#?\={2,6}')),
    'bold': (r'\1', re.compile("\'\'\'(.+?)\'\'\'")),
    'italic': (r'\1', re.compile("\'\'(.+?)\'\'")),
    'strike1': (r'\1', re.compile('~~(.+?)~~')),
    'strike2': (r'\1', re.compile('--(.+?)--')),
    'underline': (r'\1', re.compile('__(.+?)__')),
    'upper': (r'\1', re.compile('\^\^(.+?)\^\^')),
    'under': (r'\1', re.compile(',,(.+?),,')),
    
    'bigger': (r'\1', re.compile('\{\{\{\+[1-5] (.+?)\}\}\}')),
    'smaller': (r'\1', re.compile('\{\{\{\-[1-5] (.+?)\}\}\}')),
    'color': (r'\2', re.compile('\{\{\{\#(.+?) (.+?)\}\}\}')),
    'without_markup': (r'\1', re.compile('\{\{\{(.*)\}\}\}')),
    
    'macro_html': (r'\1', re.compile('\{\{\{\#\!html (.+?)\}\}\}')),
    'macro_wiki': (r'\2', re.compile('\{\{\{\#\!wiki (.+?)\n(.*)\}\}\}')),
    'macro_syntax': (r'\2', re.compile('\{\{\{\#\!syntax (.+?)\n(.*)\n\}\}\}', re.IGNORECASE)),
    'macro_color': (r'\2', re.compile('\{\{\{\#(.+?) (.+?)\}\}\}', re.IGNORECASE)),
    'macro_math': ('', re.compile('\[math\((.+?)\)\]', re.IGNORECASE)),
    'macro_date': ('', re.compile('\[date(time)?\]', re.IGNORECASE)),
    'macro_br': ('\n', re.compile('\[br\]', re.IGNORECASE)),
    'macro_include': ('', re.compile('\[include(.+?)\]', re.IGNORECASE)),
    'macro_index': ('', re.compile('\[목차\]')),
    'macro_index_': ('', re.compile('\[tableofcontents\]')),
    'macro_footnote': ('', re.compile('\[각주\]')),
    'macro_footnote_': ('', re.compile('\[footnote\]')),
    'macro_pagecount': ('', re.compile('\[pagecount(.+?)?\]', re.IGNORECASE)),
    'macro_age': ('', re.compile('\[age\(\)\]', re.IGNORECASE)),
    'macro_dday': ('', re.compile('\[dday\(\)\]', re.IGNORECASE)),
    'macro_tag': ('', re.compile('\<(.+?)\>')),

    'attach_': (r'\1', re.compile('\[\[파일:(.+?)\|(.+?)\]\]')),
    'attach': (r'\1', re.compile('\[\[파일:(.+?)\]\]')),
    
    'paragraph_': (r'\1', re.compile('\[\[#s-(.+?)\|(.+?)\]\]')),
    'paragraph': (r'\1', re.compile('\[\[#s-(.+?)\]]')),
    'link_paragraph_': (r'\1', re.compile('\[\[(.+?)#s-(.+?)\|(.+?)\]\]')),
    'link_paragraph': (r'\1', re.compile('\[\[(.+?)#s-(.+?)\]\]')),
    'link': (r'\1', re.compile('\[\[((?:(?!\|).)+?)\]\]')),
    'link_': (r'\1', re.compile('\[\[(.+?)\|(.+?)\]\]')),
    
    'list': (r'\1', re.compile('\|\*\|(.*)')),
    'list_': (r'\1', re.compile('\|\*(.*)')),
    'list__': (r'\1', re.compile('\|[1Aa]\.\|(.*)')),
    'list___': (r'\1', re.compile('\|[1Aa]\.(.*)')),
    'unordred_list': (r'\1', re.compile('[ ]+\*(.*)')),
    'ordered_list': (r'\1', re.compile('[ ]+[1AaIi]\.(.*)')),
    'quote': (r'\1', re.compile('\>+\s?(.*)')),

    'footnote': ('', re.compile('\[\*[A-Za-z]? (.+?)\]')),    
})

In [7]:
def parse(text: str, verbose: bool = False) -> str:
    def _parse(text: str, target: str, tag: re.Pattern) \
        -> Tuple[str, int]:
        return tag.subn(target, text)
    
    if verbose:
        print(f'Parsing Regex {len(regex_tags.keys())} rules\n\t{text}')
    
    while True:
        count = 0
        for key, (target, tag) in regex_tags.items():
            text, count = _parse(text, target, tag)
            if count:
                if verbose:
                    print(f'Rule [{key}: {tag}]\n\t{text}')
                break
        if not count:
            break
            
    return text.strip()

In [8]:
def parse_table(text):
    for row in regex_table.findall(text):
        values = row.split('||')
        yield values

## Get drama titles from channles

In [9]:
interested = ["JTBC 금토 드라마(2014~2017)", "JTBC 금토 드라마(2017~2020)", "JTBC 드라마", "JTBC 수목 드라마", "JTBC 월화 드라마(2011~2014)", "JTBC 월화 드라마(2017~2020)", "JTBC 주말 드라마", "KBS 수목 드라마(2001~2005)", "KBS 수목 드라마(2006~2010)", "KBS 수목 드라마(2011~2015)", "KBS 수목 드라마(2016~2020)", "KBS 월화 드라마(2001~2005)", "KBS 월화 드라마(2006~2010)", "KBS 월화 드라마(2011~2015)", "KBS 월화 드라마(2016~2020)", "KBS 학교 시리즈", "MBC 수목 미니시리즈(2006~2010)", "MBC 수목 미니시리즈(2011~2015)", "MBC 수목 미니시리즈(2016~2020)", "MBC 아침 드라마(2011~2015)", "MBC 아침 드라마(2016~2020)", "MBC 예능 드라마", "MBC 월화 미니시리즈(2006~2010)", "MBC 월화 미니시리즈(2016~2020)", "MBC 월화특별기획(2011~2015)", "MBC 일일 드라마(2016~2020)", "MBC 일일 연속극(2011~2015)", "MBC 주말 드라마(2011~2015)", "MBC 주말 드라마(2016~2020)", "MBC 주말 특별기획(2011~2015)", "MBC 주말 특별기획(2016~2020)", "MBC 하이킥 시리즈", "MBN 수목 드라마", "OCN 로맨스 드라마", "OCN 수목 오리지널", "OCN 오리지널 드라마(2010~2016)", "OCN 월화 오리지널", "OCN 토일 오리지널(2017~2020)", "SBS 금토 드라마(2019~현재)", "SBS 드라마 스페셜(1992~1995)", "SBS 드라마 스페셜(1996~2000)", "SBS 드라마 스페셜(2001~2005)", "SBS 드라마 스페셜(2006~2010)", "SBS 드라마 스페셜(2011~2015)", "SBS 드라마 스페셜(2016~2020)", "SBS 아침 연속극(2016~2020)", "SBS 월화 드라마(1991~1995)", "SBS 월화 드라마(1996~2000)", "SBS 월화 드라마(2001~2005)", "SBS 월화 드라마(2006~2010)", "SBS 월화 드라마(2011~2015)", "SBS 월화 드라마(2016~2020)", "TV CHOSUN 토일드라마", "tvN 금요 드라마(2007~2015)", "tvN 금토 드라마", "tvN 로맨스가 필요해 시리즈", "tvN 불금 시리즈(2017~)", "tvN 월화 드라마(2011~2015)", "tvN 월화 드라마(2016~2020)", "tvN 토일 드라마(2017~2020)"]

In [10]:
titles = set()

In [60]:
for inter in interested:
    document = context[inter]
    matches = regex_document.findall(document['1']['text'])
    
    for match in matches:
        name, *_ = match.split('|')
        name, *_ = name.split('#')
        titles.add(name)

## Get metadata from document

In [61]:
columns = {
    '방송 기간': ['방송기간', '방송 기간',' 방송 기간 '],
    '방송 시간': ['방송 시간', '방송시간',' 방송시간 '],
    '방송 횟수': ['횟수', '방송 횟수'],
    '장르': ['장르'],
    '채널': ['채널', '방송사'],
#    '제작사': ['제작사', '제작자', '제작'],
#    '극본': ['극본', '대본'],
#    '출연자': ['출연자', '출연', '출연진'],
}

In [62]:
data = defaultdict(dict)
notexists = []
notparser = []

In [63]:
for title in titles:
    try:
        if '(드라마)' not in title and f'{title}(드라마)' in context:
            document = context[title]['0']['text']
        else:
            document = context[title]['0']['text']
    except KeyError:
        notexists.append(title)
    d = defaultdict(str)
    for row in parse_table(parse(document)):
#         if title == '최고의 사랑' :
#             print(row)
        try:
            key, value = filter(len, row)
        except ValueError:
            continue
        key = next((ckey for ckey, cvalues in columns.items() if any(cvalue in key for cvalue in cvalues)), False)
        if key:
            key = key.strip()
            d[key] = f'{d[key]} {value}'
    
    if not d:
        notparser.append(title)
    else:
        data[title] = d

In [64]:
print(f'{len(notparser)} pages are not parsable')
print(f'{len(notexists)} pages are not exists')

125 pages are not parsable
2 pages are not exists


## Create table from data

In [65]:
table_columns = list(columns.keys())
table = np.empty((0, len(table_columns) + 1))

for title, values in data.items():
    table = np.vstack((table, np.array([
        regex_bracket.sub('', title).replace(' ', '').strip(), *tuple(map(lambda c: values[c], table_columns))
    ])))

In [66]:
df = pd.DataFrame(table)
df.columns = ['제목', *table_columns]

### Parse datetime

In [67]:
regex_date = re.compile('(.+?)년(.*)월(.*)일')

In [68]:
date_start = []
for index, date in enumerate(df['방송 기간']):
    ds, *de = map(regex_date.findall, map(str.strip, date.split('~' if '~' in date else '-')))
    ds, *_ = ds or ['unknown']
    try:
        date_start.append(pd.datetime(*tuple(map(int, ds))))
    except (ValueError, TypeError):
        date_start.append('unknown')
    
assert len(date_start) == np.size(df, 0)

In [69]:
df['방송 시작'] = pd.Series(date_start)

## Show Dataframe

In [70]:
print(df.shape)
df.head()

(736, 7)


,제목,방송 기간,방송 시간,방송 횟수,장르,채널,방송 시작
0,그남자오수,2018년 3월 5일 ~ 2018년 4월 24일,"월, 화 21:00~22:00",16부작,로맨스 범죄,OCN,2018-03-05 00:00:00
1,간택-여인들의전쟁,2019년 12월 14일 ~ 2020년 2월 9일,"토요일, 일요일 오후 10:50 ~",16부작,"로맨스, 퓨전사극",TV CHOSUN,2019-12-14 00:00:00
2,우리가만난기적,2018년 4월 2일 ~ 2018년 5월 29일,"월요일, 화요일 오후 10:00 ~ 11:10",18부작,,KBS 2TV,2018-04-02 00:00:00
3,왕과나,2007년 8월 27일 ~ 2008년 4월 1일,"월요일, 화요일 밤 9:55 ~ 11:05",63부작,,SBS,2007-08-27 00:00:00
4,굿캐스팅,2020년 4월 ~ (예정),"월요일, 화요일 오후 10:00 ~ 11:10",,,SBS,unknown


In [71]:
df.to_csv('../results/namuwiki.csv', index=None)